In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome(executable_path="C:/Users/jake_/Downloads/chrome_driver/chromedriver.exe")

In [3]:
df = pd.read_excel("C:/Users/jake_/Desktop/d3 folder/vendors_address.xlsx")


def searchPlace(sourceAddress, destinationAddress, driver):
    Source = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[2]/div/div[3]/div[1]/div[1]/div[2]/div/div/input")
    Source.send_keys(sourceAddress)
    sleep(1)
    Destination = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[2]/div/div[3]/div[1]/div[2]/div[2]/div/div/input")
    Destination.send_keys(destinationAddress)
    Destination.send_keys(Keys.ENTER)
    try:
        sleep(5) #this is going to depend on latency of network/server
        Miles_text = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[5]/div[1]/div/div[1]/div[1]/div[2]/div")
        miles = Miles_text.text
        Time_text = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[5]/div[1]/div/div[1]/div[1]/div[1]/span[1]")
        time = Time_text.text
        Best_text = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[5]/div[1]/div/div[1]/div[2]/h1[1]/span")
        best = Best_text.text
        Source.clear()
        Destination.clear()
        return [sourceAddress, destinationAddress, miles, time, best]
    except:
        Source.clear()
        Destination.clear()
        return [sourceAddress,destinationAddress,'ERROR','ERROR','ERROR']
        
    

In [4]:
def shortestPathXLSX(driver):
  #Takes no parameters
  #Returns pandas df ['Vendor', 'Source Address', 'Destination Address', 'Miles','Time','Best Route Main'] * amount in XLSX
  sleep(1.5)
  milesList = []
  error_count = 0
  driver.get("https://www.google.com/maps/dir/")
  for i in range(len(df)) :
    tup = searchPlace(df.loc[i, "Start Address"], df.loc[i, "Address"], driver)
    if tup[2] != 'ERROR':
      milesList.append([df.Vendor[i],tup[0],tup[1],tup[2],tup[3],tup[4]])
    else:
      error_count += 1

  print(milesList)
  print("Errors detected: " + str(error_count))
  milesList
  dfe = pd.DataFrame(milesList, columns =['Vendor', 'Source Address', 'Destination Address', 'Miles','Time','Best Route Main']) 
  print(dfe) 
  dfe.head()
  dfe.sort_values(by="Time")
  for i in range(len(dfe)):
    a_string = dfe['Time'][i]
    numbers = []
    for word in a_string.split():
        if word.isdigit():
          numbers.append(int(word))
    if len(numbers) == 2:
        if len(str(numbers[1])) == 1:
          fixed_time = str(numbers[0]) + ':0' + str(numbers[1])
        else:
          fixed_time = str(numbers[0]) + ':' + str(numbers[1])
    else:
        if 'hr' in a_string:
          fixed_time = str(numbers[0]) + ':' + '00'
        else:
          fixed_time = '0:' + str(numbers[0])      
    dfe['Time'][i] = fixed_time
  dfe = dfe.sort_values(by="Time")
  return dfe


In [5]:
def clean_times(dfe):
    for i in range(len(dfe)):
        a_string = dfe['Time'][i]
        numbers = []
        for word in a_string.split():
            if word.isdigit():
                numbers.append(int(word))
        if len(numbers) == 2:
            if len(str(numbers[1])) == 1:
                fixed_time = str(numbers[0]) + ':0' + str(numbers[1])
            else:
                fixed_time = str(numbers[0]) + ':' + str(numbers[1])
        else:
            if 'hr' in a_string:
                fixed_time = str(numbers[0]) + ':' + '00'
            else:
                fixed_time = '0:' + str(numbers[0])      
        dfe['Time'][i] = fixed_time
    return dfe

In [6]:
def gasPrices(zip_code, start_address, driver, top):
    driver.get("https://www.gasbuddy.com/home")
    sleep(1.5)
    StationList = []
    
    giveZip = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div[1]/form/div[1]/div[1]/div[1]/input")
    giveZip.send_keys(zip_code)
    giveZip.send_keys(Keys.ENTER)
    sleep(3)


    for i in range(1,top+1):
        try:
            name = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div[1]/div[3]/div["+str(i)+"]/div/div[2]/h3/a")
        except:
            name = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div[1]/div[3]/div["+str(i)+"]/div/div[2]/h3/span[1]/a")
            
        name = name.text

        address = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div[1]/div[3]/div["+str(i)+"]/div/div[2]/div[2]")
        address = address.text
        address = address.replace('\n',', ')

        gas_price = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div[1]/div[3]/div["+str(i)+"]/div/div[4]/div/span")
        gas_price = gas_price.text

        StationList.append([name,address,gas_price])
    dfa = pd.DataFrame(StationList, columns =['Name','Address','Gas Price']) 

    driver.get("https://www.google.com/maps/dir/")
    sleep(1.5)

    StatList = []
    for i in range(0,top):
        group = searchPlace(start_address, StationList[i][1], driver)[2:4]
        miles = group[0]
        time = group[1]
        name = StationList[i][0]
        StatList.append([name,miles,time])
        

    dfb = pd.DataFrame(StatList, columns =['Name','Miles','Time'])
    df_new = pd.merge(dfa,dfb)
    df_new = clean_times(df_new)
    df_new = df_new.sort_values(by="Time")
    return df_new

In [7]:
driver = webdriver.Chrome(executable_path="C:/Users/jake_/Downloads/chrome_driver/chromedriver.exe")
shortestPathXLSX(driver).head()
gasDF = gasPrices('28625','827 E Memorial Hwy, Harmony, NC', driver, 5).sort_values(by='Gas Price')
gasDF.head()


[['Nearest McDonalds', '827 E Memorial Hwy, Harmony, NC', 'McDonalds', '10.9 miles', '16 min', 'NC-901 S and US-64 E'], ['Nearest McAlisters Deli', '827 E Memorial Hwy, Harmony, NC', 'McAlisters Deli', '32.1 miles', '34 min', 'I-40 W and I-77 S'], ['Nearest Bojangles', '827 E Memorial Hwy, Harmony, NC', 'Bojangles', '10.8 miles', '16 min', 'NC-901 S and US-64 E'], ['Nearest Sonic', '827 E Memorial Hwy, Harmony, NC', 'Sonic', '10.9 miles', '16 min', 'NC-901 S and US-64 E'], ['Nearest Wendys', '827 E Memorial Hwy, Harmony, NC', 'Wendys', '10.6 miles', '14 min', 'NC-901 S and US-64 E'], ['Nearest Hardees', '827 E Memorial Hwy, Harmony, NC', 'Hardees', '14.0 miles', '19 min', 'NC-901 S and US-64 E'], ['Nearest Burger King', '827 E Memorial Hwy, Harmony, NC', 'Burger King', '11.3 miles', '15 min', 'NC-901 S and US-64 E'], ['Nearest KFC', '827 E Memorial Hwy, Harmony, NC', 'KFC', '11.9 miles', '15 min', 'NC-901 S and US-64 E'], ['Nearest Pizza Hut', '827 E Memorial Hwy, Harmony, NC', 'Pizza 

,Name,Address,Gas Price,Miles,Time
2,Shell,"3310 Taylorsville Hwy, Statesville, NC",$3.15,13.6 miles,0:17
1,Shell,"3310 Taylorsville Hwy, Statesville, NC",$3.15,20.6 miles,0:21
0,Sunshine Express,"2225 Wilkesboro Hwy, Statesville, NC",$3.17,20.4 miles,0:22
4,Shell,"609 Mocksville Hwy, Statesville, NC",$3.19,13.6 miles,0:17
5,7-Eleven,"234 Turnersburg Hwy, Statesville, NC",$3.19,17.3 miles,0:20
